In [15]:
path_xlsx_parte = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [16]:
import sys
sys.path.append('../_amigocloud')

In [18]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [19]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [20]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [21]:
consulta = {'query': 'select * from dataset_342996 order by idd'}
seleccion = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', consulta)
datos = seleccion['data']

In [22]:
df_datos = gpd.GeoDataFrame(datos)

In [23]:
list(set(df_datos['idd']))

[215]

In [29]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry) WHERE idd in (215)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [30]:
df_lib = pd.DataFrame(data)

In [31]:
len(df_lib)

19

In [33]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[215]

In [34]:
lista_id = list(set(df_lib['id']))
lista_id

[1920,
 1924,
 1925,
 1892,
 1893,
 1894,
 1895,
 1896,
 1900,
 1904,
 1905,
 1907,
 1909,
 1910,
 1913,
 1914,
 1915,
 1916,
 1917]

In [35]:
nulos = df_lib.isnull().sum()
nulos

id                      0
idd                     0
fecha_registro          0
piloto_1                0
piloto_2                0
hora_inicio             0
hora_fin                0
labor                   0
temperatura             0
viento                  0
humedad                 0
estado_de_operacion     0
dron                    0
obs                    19
dtype: int64

In [36]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [37]:
if null_check:
    df_lib = None
else:
    None

In [38]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,1892,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
1,1893,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
2,1894,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
3,1895,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
4,1896,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
5,1900,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
6,1904,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
7,1905,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
8,1907,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None
9,1909,215,2024-10-10 04:00:00+00:00,AURELIO GARCIA,AURELIO GARCIA,2024-10-10 13:30:00+00:00,2024-10-10 17:30:00+00:00,LIBERACION,26,13,62,CONCLUIDO,MT 01,None


In [39]:
df_parte = pd.read_excel(path_xlsx_parte)

In [40]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE
0,641,2024-05-23,21.0,62,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERAZANZA - SUAREZ DAVID,L1,17.000000,...,DA 02,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
1,558,2024-06-27,26.0,124,1530,CITTCA,299,C.I.T.T.C.A,C10.3,2.490000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
2,558,2024-06-27,26.0,124,1530,CITTCA,299,C.I.T.T.C.A,C12.2,0.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
3,558,2024-06-27,26.0,124,1530,CITTCA,299,C.I.T.T.C.A,C13.1,0.370000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
4,558,2024-06-27,26.0,124,1530,CITTCA,299,C.I.T.T.C.A,C13.2,0.760000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,NaN,NaT,NaN,79,75,AGUILERA TARADELLES NELSON MARIANO,217,FLAMBOYAN--AGUILERA,L6.1,1.756755,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,1922.0,85.0,NaN,NaN
1817,NaN,NaT,NaN,79,75,AGUILERA TARADELLES NELSON MARIANO,217,FLAMBOYAN--AGUILERA,L6.2,2.029630,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,1923.0,85.0,NaN,NaN
1818,NaN,NaT,NaN,69,20262,HEVIA Y VACA PEREZ BERNABE,791,LOS MONOS--HEBIA BERNABE,L1.2,7.726068,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,1924.0,85.0,NaN,NaN
1819,NaN,NaT,NaN,69,20262,HEVIA Y VACA PEREZ BERNABE,791,LOS MONOS--HEBIA BERNABE,L20.2,2.083266,...,NaN,NaN,NaN,NaN,NaN,SOLICITUD,1925.0,85.0,NaN,NaN


In [41]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [42]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\Usuario\AppData\Local\Temp\ipykernel_24936\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [43]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [44]:
lista_idd

[215]

In [45]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 215


In [46]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 1920
se proceso ID: 1924
se proceso ID: 1925
se proceso ID: 1892
se proceso ID: 1893
se proceso ID: 1894
se proceso ID: 1895
se proceso ID: 1896
se proceso ID: 1900
se proceso ID: 1904
se proceso ID: 1905
se proceso ID: 1907
se proceso ID: 1909
se proceso ID: 1910
se proceso ID: 1913
se proceso ID: 1914
se proceso ID: 1915
se proceso ID: 1916
se proceso ID: 1917


## Copiar ids de PARTE DIARIO a PLANIFICACION

In [47]:
path_planificacion = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\PLANIFICACION.shp'
path_xlsx = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [48]:
gdf_plan = gpd.read_file(path_planificacion)
df_xlsx = pd.read_excel(path_xlsx)

In [49]:
# Recorrer cada registro de df_xlsx
for index, row in df_xlsx.iterrows():
    id_value = row['ID']
    idd_value = row['IDD']
    
    # Actualizar el campo 'idd' en gdf_plan donde coincida el campo 'id'
    gdf_plan.loc[gdf_plan['id'] == id_value, 'idd'] = idd_value

In [50]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')